# MultiAgents 
learning from Alejandro Tutorial
https://alejandro-ao.com/posts/agents/multi-agent-deep-research/


<ol>Smolagents: A minimalist, very powerful agent library that allows you to create and run multi-agent systems with a few lines of code. </ol>
<ol>Firecrawl: A robust search-and-scrape engine for LLMs to crawl, index, and extract web content.</ol>
<ol>Open models from Hugging Face to scrape and research the web.</ol>

We will be creating a multi-agent system that is coordinated by a “Coordinator Agent” that spawns multiple “Sub-Agent” instances to handle different subtasks.

!["Agents"](/mnt/data/projects/.immune/Personal/AI_Agents_Tutorial/open-deep-research-workflow-diagram.jpg)


In [1]:
# conda activate torch_gpu_dna
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_id = "Qwen/Qwen2.5-7B-Instruct"
## KimiK2 thinking cannot be downloaded so we start with Qwen. Also my GPU is Tesla T4 so I will stick to Qwen-7B.

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_4bit=True,
    device_map="auto"
)

/mnt/data/tools/miniconda3/envs/torch_gpu_dna/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/data/tools/miniconda3/envs/torch_gpu_dna/lib/python3.10/site-packages/transformers/utils/hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 4/4 [00:43<00:00, 10.86s/it]


In [2]:
print(model.device)
print(model)

cuda:0
Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear4bit(in_features=3584, out_features=3584, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear4bit(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm): Qwen2

# 1. Generating a Research Plan

In [3]:
PLANNER_SYSTEM_INSTRUCTIONS = """
You are a research planning assistant.

Your task is to produce a clear, structured research plan
for the given user query.

Requirements:
- Break the topic into major research dimensions or questions
- Identify key biological concepts, methods, and datasets
- Include both background and cutting-edge aspects
- The plan should be suitable for later decomposition into subtasks
- Do NOT write the final answer or conclusions

Output format:
- Plain text
- Use numbered sections and bullet points
- Be concise but comprehensive
- No markdown, no JSON, no code blocks
"""


In [4]:
user_query = "Research about immune cell aging using single-cell RNA-seq"
messages = [
    {"role" : "system", "content" : PLANNER_SYSTEM_INSTRUCTIONS},
    {"role" : "user", "content" : user_query},
]

# This line converts structured chat messages into a single text prompt in the 
# exact format the model was trained on.
prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True # “End the prompt where the assistant should start replying.”
    )

print(prompt)

<|im_start|>system

You are a research planning assistant.

Your task is to produce a clear, structured research plan
for the given user query.

Requirements:
- Break the topic into major research dimensions or questions
- Identify key biological concepts, methods, and datasets
- Include both background and cutting-edge aspects
- The plan should be suitable for later decomposition into subtasks
- Do NOT write the final answer or conclusions

Output format:
- Plain text
- Use numbered sections and bullet points
- Be concise but comprehensive
- No markdown, no JSON, no code blocks
<|im_end|>
<|im_start|>user
Research about immune cell aging using single-cell RNA-seq<|im_end|>
<|im_start|>assistant



In [5]:
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
print(inputs)

{'input_ids': tensor([[151644,   8948,    271,   2610,    525,    264,   3412,   9115,  17847,
            382,   7771,   3383,    374,    311,   8193,    264,   2797,     11,
          32930,   3412,   3119,    198,   1958,    279,   2661,   1196,   3239,
            382,  59202,    510,     12,  15623,    279,   8544,   1119,   3598,
           3412,  15336,    476,   4755,    198,     12,  64547,   1376,  23275,
          18940,     11,   5413,     11,    323,  29425,    198,     12,  29734,
           2176,   4004,    323,  14376,  47348,  13566,    198,     12,    576,
           3119,   1265,    387,  14452,    369,   2937,  65166,   1119,   1186,
          24760,    198,     12,   3155,   4183,   3270,    279,   1590,   4226,
            476,  30242,    271,   5097,   3561,    510,     12,  43199,   1467,
            198,     12,   5443,  48826,  14158,    323,  17432,   3501,    198,
             12,   2823,  63594,    714,  15817,    198,     12,   2308,  50494,
             1

In [6]:
def generate_research_plan(user_query: str) -> str:
    print("Generating the research plan for the query:", user_query)
    print("MODEL:", model_id)

    messages = [
        {"role": "system", "content": PLANNER_SYSTEM_INSTRUCTIONS},
        {"role": "user", "content": user_query},
    ]

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Not training do not track the gradients. Since this is inference so No Gradient
# This runs autoregressive text generation:
# Feed input tokens
# Predict next token
# Append it
# Repeat until:
# max_new_tokens reached
# EOS token generated
    with torch.no_grad(): 
        output = model.generate(
            **inputs, ## ** since model.generate expect keyword argument rather than dictionary, so ** makes it keyword
            max_new_tokens=500,
            temperature=0.3, # very focused no creativity
            do_sample=False,  # Greedy decoding, most probable token temperature does not have any effect
            repetition_penalty=1.1 # light penalty, where 1 is no penalty
        )

# Best for:
# Planning
# Structured output
# JSON
# Deterministic results

    response = tokenizer.decode(
        output[0][inputs["input_ids"].shape[-1]:],
        skip_special_tokens=True
    )

    print("\033[93mGenerated Research Plan\033[0m")
    print(f"\033[93m{response}\033[0m")

    return response.strip()

In [7]:
research_plan = generate_research_plan(
    "Research about immune cell aging using single-cell RNA-seq"
)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Generating the research plan for the query: Research about immune cell aging using single-cell RNA-seq
MODEL: Qwen/Qwen2.5-7B-Instruct
Generated Research Plan
1. Introduction to Immune Cell Aging
   - Definition of immune cell aging
   - Overview of age-related changes in immune function
2. Background on Single-Cell RNA Sequencing (scRNA-seq)
   - Principles of scRNA-seq technology
   - Advantages of scRNA-seq over bulk RNA sequencing
3. Key Biological Concepts Related to Immune Cell Aging
   - Senescence and exhaustion markers in immune cells
   - Epigenetic changes associated with aging
   - Intrinsic vs extrinsic factors influencing immune aging
4. Research Questions
   - How do gene expression patterns change during immune cell aging?
   - What specific cellular pathways are affected by aging?
   - Can we identify unique transcriptional signatures of aged immune cells?
5. Methods for Studying Immune Cell Aging Using scRNA-seq
   - Sample collection and preparation techniques
   - D

<h4> Kimi-k2 </h4>
So previous model Kimi-k2 thinking has much better thinking so we can provide long instruction
Kimi-K2-Thinking (and similar “reasoning” models) has:
<ol>Strong instruction-following</ol>
<ol> Hidden chain-of-thought / internal planning</ol>
<ol>Better tolerance for long, nuanced constraints</ol>

So this worked well:
<ol>Rich requirements</ol>
<ol>Soft heuristics (“use your judgment”)</ol>
<ol>Multi-objective planning</ol>

The KimiK2 models could:
<p>Think → structure → output JSON </p>

<h4> Qwen model </h4>
What changes with local Transformers (Qwen2.5-7B-Instruct)
<ol>4-bit</ol>
<ol>no reasoning mode</ol>
<ol>no response_format enforcement</ol>

This means:
<ol>Risks with long instructions</ol>
<ol>Model may explain itself</ol>
<ol>Model may summarize constraints</ol>
<ol>Model may violate JSON-only</ol>
<ol>Model may partially follow constraints</ol>

But…
<ol>Benefits of long instructions</ol>
<ol>Better task decomposition</ol>
<ol>Better coverage</ol>
<ol>Less shallow subtasks</ol>

# 2. Dividing into sub task
Each Agent or subtask would help the agent to take the action
<h4> shorter instruction to Qwen

In [8]:
# Json expected from the LLM which can be done by pydantic
{
  "subtasks": [
    {
      "id": "cell_types",
      "title": "Immune cell types affected by aging",
      "description": "..."
    },
    {
      "id": "pathways",
      "title": "Aging-associated pathways",
      "description": "..."
    }
  ]
}


{'subtasks': [{'id': 'cell_types',
   'title': 'Immune cell types affected by aging',
   'description': '...'},
  {'id': 'pathways',
   'title': 'Aging-associated pathways',
   'description': '...'}]}

In [9]:
import json
from pydantic import BaseModel, Field
from typing import List
from pprint import pprint

## This create a structed json that would be feed into LLM for subtask for each agent
class Subtask(BaseModel): # subtask inherits from BaseModel i.e. from pydantic to make it in a json format
    id: str = Field(
        ...,
        description="Short identifier for the subtask (e.g. 'A', 'history', 'drivers').",
    )
    title: str = Field(
        ...,
        description="Short descriptive title of the subtask.",
    )
    description: str = Field(
        ...,
        description="Clear, detailed instructions for the sub-agent that will research this subtask.",
    )

class SubtaskList(BaseModel):
    subtasks: List[Subtask] = Field(
        ...,
        description="List of subtasks that together cover the whole research plan.",
    )


In [10]:
TASK_SPLITTER_SYSTEM_INSTRUCTIONS = f"""
You will be given a research plan.

Your job is to split it into subtasks.

Return ONLY valid JSON in the following schema:

{json.dumps(SubtaskList.model_json_schema(), indent=2)}

Rules:
- Do not include any explanation
- Do not include markdown
- Do not include text outside JSON
- Output must be valid JSON
"""
# (SubtaskList.model_json_schema It converts your Python data model into a formal JSON Schema.
# 1️⃣ model_json_schema()
# Returns a Python dict.
# 2️⃣ json.dumps(...)
# Converts that dict → JSON string.
# 3️⃣ indent=2
# Pretty-prints it so:
# Humans can read it
# LLMs parse it more accurately

## Local Generation Transformers

In [11]:
# We have create these json into the tokenizer form to feed into the transformers
def generate_json_response(prompt: str, max_new_tokens=1024):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.2,
            do_sample=False,
            eos_token_id=tokenizer.eos_token_id,
        )

    output_text = tokenizer.decode(
        output_ids[0][inputs["input_ids"].shape[-1]:],
        skip_special_tokens=True,
    )

    return output_text.strip()

#### JSON extraction + Pydantic validation
LLMs sometimes add junk — extract safely.

In [12]:
import re

def extract_json(text: str) -> str:
    start = text.find("{")
    if start == -1:
        raise ValueError("No JSON object found")

    brace_count = 0
    for i in range(start, len(text)):
        if text[i] == "{":
            brace_count += 1
        elif text[i] == "}":
            brace_count -= 1
            if brace_count == 0:
                return text[start:i + 1]

    raise ValueError("Unbalanced JSON braces")


In [13]:
prompt = f"""
{TASK_SPLITTER_SYSTEM_INSTRUCTIONS}

Research plan:
{research_plan}
"""

In [14]:
import pprint
pprint.pprint(prompt)

('\n'
 '\n'
 'You will be given a research plan.\n'
 '\n'
 'Your job is to split it into subtasks.\n'
 '\n'
 'Return ONLY valid JSON in the following schema:\n'
 '\n'
 '{\n'
 '  "$defs": {\n'
 '    "Subtask": {\n'
 '      "properties": {\n'
 '        "id": {\n'
 '          "description": "Short identifier for the subtask (e.g. \'A\', '
 '\'history\', \'drivers\').",\n'
 '          "title": "Id",\n'
 '          "type": "string"\n'
 '        },\n'
 '        "title": {\n'
 '          "description": "Short descriptive title of the subtask.",\n'
 '          "title": "Title",\n'
 '          "type": "string"\n'
 '        },\n'
 '        "description": {\n'
 '          "description": "Clear, detailed instructions for the sub-agent '
 'that will research this subtask.",\n'
 '          "title": "Description",\n'
 '          "type": "string"\n'
 '        }\n'
 '      },\n'
 '      "required": [\n'
 '        "id",\n'
 '        "title",\n'
 '        "description"\n'
 '      ],\n'
 '      "title": "

In [15]:
raw_output = generate_json_response(prompt)
print(raw_output)

```json
{
  "subtasks": [
    {
      "id": "A",
      "title": "Introduction to Immune Cell Aging",
      "description": "Define immune cell aging and overview age-related changes in immune function."
    },
    {
      "id": "B",
      "title": "Background on Single-Cell RNA Sequencing",
      "description": "Explain principles of scRNA-seq technology and its advantages over bulk RNA sequencing."
    },
    {
      "id": "C",
      "title": "Key Biological Concepts",
      "description": "Identify senescence and exhaustion markers, epigenetic changes, and intrinsic vs extrinsic factors influencing immune aging."
    },
    {
      "id": "D",
      "title": "Research Questions",
      "description": "Formulate questions about gene expression patterns, affected cellular pathways, and transcriptional signatures of aged immune cells."
    },
    {
      "id": "E",
      "title": "Methods for Studying Immune Cell Aging",
      "description": "Detail sample collection, preparation, normali

In [16]:
json_text = extract_json(raw_output)
print(json_text)

{
  "subtasks": [
    {
      "id": "A",
      "title": "Introduction to Immune Cell Aging",
      "description": "Define immune cell aging and overview age-related changes in immune function."
    },
    {
      "id": "B",
      "title": "Background on Single-Cell RNA Sequencing",
      "description": "Explain principles of scRNA-seq technology and its advantages over bulk RNA sequencing."
    },
    {
      "id": "C",
      "title": "Key Biological Concepts",
      "description": "Identify senescence and exhaustion markers, epigenetic changes, and intrinsic vs extrinsic factors influencing immune aging."
    },
    {
      "id": "D",
      "title": "Research Questions",
      "description": "Formulate questions about gene expression patterns, affected cellular pathways, and transcriptional signatures of aged immune cells."
    },
    {
      "id": "E",
      "title": "Methods for Studying Immune Cell Aging",
      "description": "Detail sample collection, preparation, normalization, 

In [17]:
data = json.loads(json_text)
pprint.pprint(data)

{'subtasks': [{'description': 'Define immune cell aging and overview '
                              'age-related changes in immune function.',
               'id': 'A',
               'title': 'Introduction to Immune Cell Aging'},
              {'description': 'Explain principles of scRNA-seq technology and '
                              'its advantages over bulk RNA sequencing.',
               'id': 'B',
               'title': 'Background on Single-Cell RNA Sequencing'},
              {'description': 'Identify senescence and exhaustion markers, '
                              'epigenetic changes, and intrinsic vs extrinsic '
                              'factors influencing immune aging.',
               'id': 'C',
               'title': 'Key Biological Concepts'},
              {'description': 'Formulate questions about gene expression '
                              'patterns, affected cellular pathways, and '
                              'transcriptional signatures of aged i

In [18]:
subtask_list = SubtaskList(**data)

In [19]:
subtask_list.subtasks

[Subtask(id='A', title='Introduction to Immune Cell Aging', description='Define immune cell aging and overview age-related changes in immune function.'),
 Subtask(id='B', title='Background on Single-Cell RNA Sequencing', description='Explain principles of scRNA-seq technology and its advantages over bulk RNA sequencing.'),
 Subtask(id='C', title='Key Biological Concepts', description='Identify senescence and exhaustion markers, epigenetic changes, and intrinsic vs extrinsic factors influencing immune aging.'),
 Subtask(id='D', title='Research Questions', description='Formulate questions about gene expression patterns, affected cellular pathways, and transcriptional signatures of aged immune cells.'),
 Subtask(id='E', title='Methods for Studying Immune Cell Aging', description='Detail sample collection, preparation, normalization, quality control, clustering, and differential expression analysis.'),
 Subtask(id='F', title='Datasets and Resources', description='Locate publicly available 

In [23]:
def split_into_subtasks(research_plan: str):

    prompt = f"""
{TASK_SPLITTER_SYSTEM_INSTRUCTIONS}

Research plan:
{research_plan}
"""

    raw_output = generate_json_response(prompt)

    json_text = extract_json(raw_output)
    data = json.loads(json_text)

    # 🔒 Validate with Pydantic
    subtask_list = SubtaskList(**data)

    print("\033[93mGenerated The Following Subtasks\033[0m")
    for task in subtask_list.subtasks:
        print(f"\033[93m{task.title}\033[0m")
        print(task.description)
        print()

    return subtask_list.subtasks


In [22]:
subtasks = split_into_subtasks(research_plan)


Generated The Following Subtasks
Introduction to Immune Cell Aging


TypeError: 'module' object is not callable

In [24]:
import json
TASK_SPLITTER_SYSTEM_INSTRUCTIONS = f"""
You are a task decomposition engine.

You will be given a set of research instructions (a research plan).
Your job is to break this plan into a set of coherent, non-overlapping
subtasks that can be researched independently by separate agents.

Planning guidelines:
- 3 to 8 subtasks is usually a good range. Use your judgment.
- Subtasks should collectively cover the full scope of the original plan
  without unnecessary duplication.
- Prefer grouping by meaningful dimensions such as:
  time periods, regions, actors, themes, or causal mechanisms,
  depending on the topic.
- Do NOT include a final task that synthesizes results.
  That will be done later in another step.
- Each subtask description should be very clear and detailed about
  what the agent must research and produce.

Output requirements (STRICT):
- Return ONLY valid JSON
- Do NOT include explanations
- Do NOT include markdown
- Do NOT include text outside JSON
- Output MUST conform exactly to the following schema:

{json.dumps(SubtaskList.model_json_schema(), indent=2)}
"""


In [25]:
subtasks_long = split_into_subtasks(research_plan)

Generated The Following Subtasks
Definition and Overview of Immune Cell Aging
Research the definition of immune cell aging and provide an overview of age-related changes in immune function. This includes identifying key terms, concepts, and mechanisms involved in the aging process of immune cells.

Introduction to Single-Cell RNA Sequencing Technology
Examine the principles of single-cell RNA sequencing (scRNA-seq) technology, including its methodology, advantages over bulk RNA sequencing, and how it can be used to study immune cell aging at the individual cell level.

Key Biological Concepts Related to Immune Cell Aging
Investigate senescence and exhaustion markers in immune cells, epigenetic changes associated with aging, and the distinction between intrinsic and extrinsic factors influencing immune aging. Provide a comprehensive understanding of these concepts and their relevance to the aging process.

Research Questions on Immune Cell Aging
Formulate specific research questions reg

## Create subagents + coordinator

In this step, we’ll create a tool that spins up a dedicated sub-agent for each subtask. This tool will be handed to the Coordinator agent, which will invoke it whenever a new subtask needs to be processed. Each sub-agent will perform thorough research on its assigned subtask and return its findings once completed. The Coordinator will then aggregate all sub-agent outputs into a comprehensive deep-research report.

In [ ]:
SUBAGENT_PROMPT_TEMPLATE = """
You are a specialized computational biology sub-agent.

Global research question:
{user_query}

Overall research plan:
{research_plan}

Your assigned subtask:
ID: {subtask_id}
Title: {subtask_title}

Task description:
\"\"\"{subtask_description}\"\"\"

Instructions:
- Focus ONLY on this subtask.
- Reason using domain knowledge in single-cell RNA-seq and immunology.
- When relevant, propose:
  • analysis strategies
  • statistical tests
  • potential confounders
  • biological interpretation
- Be explicit about assumptions and limitations.
- If relevant, suggest datasets, tools, or methods.

Return a MARKDOWN report with:

# [{subtask_id}] {subtask_title}

## Objective
What this subtask aims to resolve.

## Proposed Approach
Methods, tools, or analyses to use.

## Expected Results
What outcomes would support or refute hypotheses.

## Caveats & Risks
Batch effects, annotation bias, etc.

## References (optional)
Key papers or resources (no browsing required).
"""


In [ ]:
subagent = ToolCallingAgent(
    tools=[],
    model=subagent_model,
    add_base_tools=False,
    name=f"subagent_{subtask_id}",
)


In [26]:
DATA_CURATION_AGENT_PROMPT = """
You are a DATA CURATION AGENT specializing in single-cell RNA-seq.

Global research question:
{user_query}

Your task:
Identify publicly available scRNA-seq datasets related to immune cell aging.

Criteria:
- Study must involve aging (young vs old or lifespan comparisons)
- Must include immune cells (PBMCs, T cells, B cells, myeloid, etc.)
- Prefer datasets with PROCESSED single-cell files:
  • .h5ad
  • .h5Seurat
  • .rds
  • .loom
  • .tsv
  • .csv
- Organism: human or mouse

For each candidate dataset, report:

## Dataset <number>
- Study title
- Accession ID (e.g., GSEXXXXX)
- Organism
- Immune cell types included
- Age groups
- scRNA-seq platform
- Available processed file formats
- Why this dataset is useful for immune aging analysis

After listing datasets, include a final section:

## User Action Required
Ask the user:
- Which dataset(s) should be downloaded?
- Preferred file format if multiple are available?

Do NOT download anything yet.
Return ONLY a markdown report.
"""


In [35]:
def run_local_agent(prompt: str, max_new_tokens=1024):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.2,
            do_sample=False,
            repetition_penalty=1.1,
            eos_token_id=tokenizer.eos_token_id,
        )

    text = tokenizer.decode(
        output_ids[0][inputs["input_ids"].shape[-1]:],
        skip_special_tokens=True,
    )
    return text.strip()


In [36]:
report = run_local_agent(DATA_CURATION_AGENT_PROMPT)
print(report)


```markdown
# Datasets for Single Cell RNA Sequencing of Immune Cells during Aging

## Dataset 1
- **Study Title:** "Single-Cell Transcriptomic Analysis of Human Immune Cells from Young and Old Individuals"
- **Accession ID:** GSE12345
- **Organism:** Human
- **Immune Cell Types Included:** PBMCs, T cells, B cells, myeloid cells
- **Age Groups:** Young (20-30 years), Old (60-70 years)
- **scRNA-seq Platform:** 10x Genomics
- **Available Processed File Formats:** `.h5ad`, `.loom`
- **Why This Dataset is Useful for Immune Aging Analysis:** Comprehensive analysis of various immune cell types across different age groups using high-quality processed data.

## Dataset 2
- **Study Title:** "Temporal Dynamics of Murine Immune Cells During Aging"
- **Accession ID:** GSM67890
- **Organism:** Mouse
- **Immune Cell Types Included:** CD4+ T cells, CD8+ T cells, B cells
- **Age Groups:** Juvenile (3 months), Adult (12 months), Senescent (24 months)
- **scRNA-seq Platform:** Drop-seq
- **Available Pr